In [1]:
import pandas as pd
import numpy as np
import statistics
import itertools
pd.options.mode.chained_assignment = None

In [2]:
filename = "dados/ipca_hist.xlsx"

In [3]:
df = pd.read_excel(filename)

In [4]:
df.set_index('Data', inplace=True)

In [5]:
df_ipca = df.loc[:, 'IPCA a.a.':'IPCA a.m.']

In [6]:
df_r = df.iloc[:, :-2]

In [7]:
# Log Retorno
df_r = np.log(df_r) - np.log(df_r.shift(1))

In [8]:
# Calculando Retorno Real
df_rr = (df_r + 1).div((df_ipca['IPCA a.m.'].shift(1) + 1), axis=0) - 1

In [9]:
df_rr = df_rr[1:]

In [47]:
df_rr.head()

,VALE3,ITUB4,B3SA3,PETR4,BBDC4,PETR3,ABEV3,BBAS3,MGLU3,ITSA4,JBSS3,LREN3,WEGE3,BBDC3,RENT3,RADL3,IBOV
Data,,,,,,,,,,,,,,,,,
2014-02-01,-0.006030,0.027184,0.044096,-0.082681,-0.056579,-0.062572,0.049068,-0.013618,-0.125538,0.005324,-0.123247,0.035432,0.105109,0.012220,0.024833,0.123856,-0.015977
2014-03-01,-0.053431,0.078099,0.105705,0.144129,0.241514,0.135958,-0.000473,0.097971,-0.011556,0.083096,0.029330,0.101975,0.061737,0.138575,0.056426,0.166499,0.063237
2014-04-01,-0.066018,0.077444,0.008218,0.043046,-0.035450,0.041812,-0.042556,0.036120,0.153676,0.053085,-0.011393,0.030274,0.102022,0.018676,0.001623,-0.036118,0.018658
2014-05-01,-0.014775,-0.058968,-0.042410,0.066568,0.029049,0.026538,-0.030026,-0.026562,0.192679,-0.043963,-0.032538,0.113646,-0.038912,-0.070626,0.063769,-0.005063,-0.012549
2014-06-01,0.021766,0.005492,0.058162,0.030004,-0.072246,0.034230,-0.011425,0.080004,0.115570,0.011025,0.017997,0.034285,0.078470,-0.000913,0.039328,-0.045745,0.031633


# Funções

In [10]:
def getMinVarPesos(retornos):
    pesos = {}
    
    #(Xi - Xbarra)
    rm_rp = retornos - retornos.mean()
    
    # (Xi - Xbarra)^2
    cov = rm_rp.T @ rm_rp
    
    # ((Xi - Xbarra)^2)/(n-1)
    cov_og = cov/(retornos.count()[0] - 1)
    
    # Construíndo matriz
    cov_2 = cov_og*2
    cov_2["1"] = 1
    l1 = []
    for i in range(1, cov_2.shape[1]):
        l1.append(1)
    l1.append(0)
    
    ## MIN VARIÂNCIA GLOBAL
    l1_series = pd.Series(l1, index = cov_2.columns)
    cov_2 = cov_2.append(l1_series, ignore_index=True)
    
    _b = np.array(abs(cov_2["1"] - 1))
    
    x = np.linalg.solve(cov_2, _b)
    
    pesos_vals = x[:-1]
    
    for idx, peso in enumerate(x[:-1]):
        pesos[cov_2.keys()[:-1][idx]] = peso
        
    # Retorno Esperado
    re = pesos_vals.dot(retornos.mean())
    
    # Variância
    var = pesos_vals.dot(cov_2.iloc[:-1,:-1].dot(pesos_vals.T))
    
    # Desvio Padrão
    dp = var**0.5
    
    # Índice Sharpe
    sharpe = round((re)/dp, 4)
    
    return pesos, sharpe

In [11]:
def getMaxSharpeMin(ativos, minAtivos, index, verbose):
    rel = {}
    max_sharpe = -9999
    max_comb = ()
    max_pesos = pd.DataFrame()
    for i in range(minAtivos, len(ativos) + 1):
        for comb in itertools.combinations(ativos, i):
            pesos, sharpe = getMinVarPesos(df_rr[list(comb)][index-12:index])
            rel[comb] = sharpe 
            if(max_sharpe < sharpe):
                max_sharpe = sharpe
                max_comb = comb
                max_pesos = pesos
        if(verbose):
            print("Combinação terminada: " + str(i))
    return max_sharpe, max_comb, max_pesos

In [12]:
def getRetorno(row, tipo):
    r = []
    if(tipo == 'GlobalMinFixo'):
        for key, value in row['GlobalMinPesos'].items():
            r.append((np.exp(row[key])-1) * value)
    elif(tipo == 'GlobalMinMovel'):
        for key, value in row['GlobalMinPesosMod'].items():
            r.append((np.exp(row[key])-1) * value)
    elif(tipo == 'GlobalMinMovel2'):
        for key, value in row['GlobalMinPesosMod2'].items():
            r.append((np.exp(row[key])-1) * value)
    return np.log(sum(r)+1)

### Carteira 1 (Composição Fixa)

In [13]:
# Seleção de ativos para compor a carteira
ativos = ['VALE3', 'ITUB4', 'B3SA3', 'PETR4', 'BBDC4']

In [14]:
# Definindo os pesos para cada mês
pesos_fixo = []
for j in range(12, df_rr.shape[0]):
    max_sharpe, max_combo, max_pesos = getMaxSharpeMin(ativos, 5, j, False)
    pesos_fixo.append(max_pesos)

In [15]:
df_rr_copy = df_rr.copy()
df_valid = df_rr_copy[12:]

In [16]:
df_valid['GlobalMinPesos'] = pesos_fixo

In [17]:
# Obtendo os retornos mensais com base na composição de pesos da carteira
df_valid['Carteira Fixa'] = df_valid.apply(lambda row : getRetorno(row, 'GlobalMinFixo'), axis = 1)

In [18]:
# Calculando o retorno acumulado
df_valid['Retorno Acumulado Carteira Fixa'] = np.cumsum(df_valid['Carteira Fixa'].values)
df_valid['Retorno Acumulado IBOV'] = np.cumsum(df_valid['IBOV'].values)

In [19]:
df_valid['Carteira Fixa'] = round(df_valid['Carteira Fixa']*100, 4)
df_valid['Retorno Acumulado Carteira Fixa'] = round(df_valid['Retorno Acumulado Carteira Fixa']*100, 4)
df_valid['IBOV'] = round(df_valid['IBOV']*100, 4)
df_valid['Retorno Acumulado IBOV'] = round(df_valid['Retorno Acumulado IBOV']*100, 4)

### Plot

In [20]:
import plotly.express as px

In [21]:
df_valid[['Carteira Fixa', 'IBOV']].describe()

,Carteira Fixa,IBOV
count,64.000000,64.000000
mean,1.783500,0.518941
std,11.207958,7.383174
min,-34.278000,-35.741900
25%,-3.223625,-2.728825
50%,1.819400,0.378450
75%,7.872650,5.069575
max,45.264800,14.726100


In [70]:
fig = px.line(df_valid[['IBOV', 'Carteira Fixa']],
             title="Retorno Mensal Carteira Fixa x IBOV",
             labels={
                 "variable": "Ativo",  "value": "Retorno Mensal (%)"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [48]:
df_plot_acum = df_valid[['Retorno Acumulado IBOV', 'Retorno Acumulado Carteira Fixa']].copy()

In [49]:
df_plot_acum.rename(columns={'Retorno Acumulado IBOV':'IBOV',
                            'Retorno Acumulado Carteira Fixa': 'Carteira Fixa'}, 
                 inplace=True)

In [50]:
fig_acum = px.line(df_plot_acum,
             title="Retorno Acumulado Carteira Fixa",
             labels={
                 "variable": "Ativo",  "value": "Retorno Acumulado (%)"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig_acum.show()

## CARTEIRA 2 (Otimizando composição por Sharpe)

In [26]:
# Seleção de ativos para compor a carteira
ativos = ['VALE3', 'ITUB4', 'B3SA3', 'PETR4', 'BBDC4']

In [27]:
# Definindo os pesos para cada mês
results = []
for j in range(12, df_r.shape[0] - 1):
    max_sharpe, max_combo, max_pesos = getMaxSharpeMin(ativos, 4, j, False)
    results.append(max_pesos)

In [28]:
df_valid['GlobalMinPesosMod'] = results

In [29]:
# Obtendo os retornos mensais e acumulados com base na composição de pesos da carteira
df_valid['Carteira Móvel'] = df_valid.apply(lambda row : getRetorno(row, 'GlobalMinMovel'), axis = 1)
df_valid['Retorno Acumulado Carteira Móvel'] = np.cumsum(df_valid['Carteira Móvel'].values)

In [30]:
df_valid['Carteira Móvel'] = round(df_valid['Carteira Móvel']*100, 4)
df_valid['Retorno Acumulado Carteira Móvel'] = round(df_valid['Retorno Acumulado Carteira Móvel']*100, 4)

## Plot

In [31]:
df_valid[['Carteira Móvel', 'Carteira Fixa', 'IBOV']].describe()

,Carteira Móvel,Carteira Fixa,IBOV
count,64.000000,64.000000,64.000000
mean,1.667488,1.783500,0.518941
std,10.763488,11.207958,7.383174
min,-32.089800,-34.278000,-35.741900
25%,-3.018300,-3.223625,-2.728825
50%,1.814600,1.819400,0.378450
75%,6.038075,7.872650,5.069575
max,37.218500,45.264800,14.726100


In [71]:
fig_m = px.line(df_valid[['IBOV', 'Carteira Móvel']],
             title="Retorno Mensal Carteira Móvel",
             labels={
                 "variable": "Ativo",  "value": "Retorno Mensal (%)"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig_m.show()

In [51]:
df_plot_acum_2 = df_valid[['Retorno Acumulado IBOV', 'Retorno Acumulado Carteira Fixa',
                           'Retorno Acumulado Carteira Móvel']].copy()

In [52]:
df_plot_acum_2.rename(columns={'Retorno Acumulado IBOV': 'IBOV', 'Retorno Acumulado Carteira Fixa': 'Carteira Fixa',
                               'Retorno Acumulado Carteira Móvel': 'Carteira Móvel'}, 
                 inplace=True)

In [72]:
fig_m_acum = px.line(df_plot_acum_2,
             title="Retorno Acumulado Carteira Móvel",
             labels={
                 "variable": "Ativo",  "value": "Retorno Acumulado (%)"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig_m_acum.show()

## Carteira 3 (Composição Móvel com mais ativos)

In [36]:
# Seleção de ativos para compor a carteira
ativos_mov = ['VALE3', 'ITUB4', 'B3SA3', 'PETR4', 'BBDC4', 'ABEV3', 'BBAS3', 'MGLU3']

In [37]:
# Definindo os pesos para cada mês
results_all = []
for j in range(12, df_r.shape[0] - 1):
    max_sharpe, max_combo, max_pesos = getMaxSharpeMin(ativos_mov, 4, j, True)
    print("Done: " + str(j))
    results_all.append(max_pesos)

Combinação terminada: 4
Combinação terminada: 5
Combinação terminada: 6
Combinação terminada: 7
Combinação terminada: 8
Done: 12
Combinação terminada: 4
Combinação terminada: 5
Combinação terminada: 6
Combinação terminada: 7
Combinação terminada: 8
Done: 13
Combinação terminada: 4
Combinação terminada: 5
Combinação terminada: 6
Combinação terminada: 7
Combinação terminada: 8
Done: 14
Combinação terminada: 4
Combinação terminada: 5
Combinação terminada: 6
Combinação terminada: 7
Combinação terminada: 8
Done: 15
Combinação terminada: 4
Combinação terminada: 5
Combinação terminada: 6
Combinação terminada: 7
Combinação terminada: 8
Done: 16
Combinação terminada: 4
Combinação terminada: 5
Combinação terminada: 6
Combinação terminada: 7
Combinação terminada: 8
Done: 17
Combinação terminada: 4
Combinação terminada: 5
Combinação terminada: 6
Combinação terminada: 7
Combinação terminada: 8
Done: 18
Combinação terminada: 4
Combinação terminada: 5
Combinação terminada: 6
Combinação terminada: 7
C

In [38]:
df_valid['GlobalMinPesosMod2'] = results_all

In [39]:
# Obtendo os retornos mensais e acumulados com base na composição de pesos da carteira
df_valid['Carteira Móvel Mod'] = df_valid.apply(lambda row : getRetorno(row, 'GlobalMinMovel2'), axis = 1)
df_valid['Retorno Acumulado Carteira Móvel Modificada'] = np.cumsum(df_valid['Carteira Móvel Mod'].values)

In [40]:
df_valid['Carteira Móvel Mod'] = round(df_valid['Carteira Móvel Mod']*100, 4)
df_valid['Retorno Acumulado Carteira Móvel Modificada'] = round(df_valid['Retorno Acumulado Carteira Móvel Modificada']*100, 4)

### Plot

In [41]:
df_valid[['Carteira Móvel Mod', 'Carteira Móvel', 'Carteira Fixa', 'IBOV']].describe()

,Carteira Móvel Mod,Carteira Móvel,Carteira Fixa,IBOV
count,64.000000,64.000000,64.000000,64.000000
mean,3.294606,1.667488,1.783500,0.518941
std,11.494765,10.763488,11.207958,7.383174
min,-28.269500,-32.089800,-34.278000,-35.741900
25%,-3.372925,-3.018300,-3.223625,-2.728825
50%,3.061400,1.814600,1.819400,0.378450
75%,7.560275,6.038075,7.872650,5.069575
max,37.410700,37.218500,45.264800,14.726100


In [73]:
fig_m2 = px.line(df_valid[['IBOV', 'Carteira Móvel Mod']],
             title="Retorno Mensal Carteira Móvel Modificada",
             labels={
                 "variable": "Ativo",  "value": "Retorno Mensal (%)"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig_m2.show()

In [54]:
df_plot_acum_3 = df_valid[['Retorno Acumulado IBOV', 'Retorno Acumulado Carteira Fixa','Retorno Acumulado Carteira Móvel', 
                           'Retorno Acumulado Carteira Móvel Modificada']].copy()

In [55]:
df_plot_acum_3.rename(columns={'Retorno Acumulado IBOV': 'IBOV', 'Retorno Acumulado Carteira Fixa': 'Carteira Fixa',
                               'Retorno Acumulado Carteira Móvel': 'Carteira Móvel', 
                               'Retorno Acumulado Carteira Móvel Modificada': 'Carteira Móvel 2'}, 
                 inplace=True)

In [74]:
fig_m_acum = px.line(df_plot_acum_3,
             title="Retorno Acumulado Carteira Móvel Modificada",
             labels={
                 "variable": "Ativo",  "value": "Retorno Acumulado (%)"
             })
# fig.update_xaxes(rangeslider_visible=True)
fig_m_acum.show()